In [1]:
from __future__ import division, print_function
import sys
import os
import json
import sys
import csv
import pandas as pd
import numpy as np
import pylab as pl
from pandas import Series, DataFrame
%pylab inline
#import seaborn
from scipy.optimize import curve_fit, minimize

Populating the interactive namespace from numpy and matplotlib


In [2]:
if os.getenv ('PUI2016') is None:
    print ("Must set env variable PUI2016")
print (os.getenv ('PUI2016'))
print (os.getenv ('PUIDATA'))

/home/cusp/rav220/PUI2016
/home/cusp/rav220/PUIdata


In [3]:
def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    # check if not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        # otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            # unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            # NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
                os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    # one final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")
        
print('ok?')

ok?


In [4]:
datestring = '201402'
getCitiBikeCSV(datestring)

file in place, you can continue


In [5]:
dataf = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
len(dataf)

224736

In [6]:
dataf['date'] = pd.to_datetime(dataf['starttime'])
dataf.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,21101,Subscriber,1991,1,2014-02-01 00:00:00
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,-73.989780,15456,Subscriber,1979,2,2014-02-01 00:00:03
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,-73.994800,16281,Subscriber,1948,2,2014-02-01 00:00:09
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,17400,Subscriber,1981,1,2014-02-01 00:00:32
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,-73.989780,19341,Subscriber,1990,1,2014-02-01 00:00:41


In [7]:
dataf.dropna(how='any', inplace = 'true')

In [8]:
len(dataf)

224736

In [9]:
dataf.columns

Index([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender', u'date'],
      dtype='object')

In [10]:
dataf['ageM'] = 2014 - dataf['birth year'][(dataf['usertype'] == 'Subscriber') & (dataf['gender'] == 1)]
dataf['ageF'] = 2014 - dataf['birth year'][(dataf['usertype'] == 'Subscriber') & (dataf['gender'] == 2)]

TypeError: unsupported operand type(s) for -: 'int' and 'str'

In [11]:
dataf.drop(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid'], axis = 1, inplace = True)
dataf.head()

,usertype,birth year,gender,date
0,Subscriber,1991,1,2014-02-01 00:00:00
1,Subscriber,1979,2,2014-02-01 00:00:03
2,Subscriber,1948,2,2014-02-01 00:00:09
3,Subscriber,1981,1,2014-02-01 00:00:32
4,Subscriber,1990,1,2014-02-01 00:00:41


In [12]:
dataf.describe()

,gender
count,224736.000000
mean,1.154617
std,0.436592
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [13]:
dataf['ageM'].dropna(inplace= True)
dataf['ageF'].dropna(inplace= True)

KeyError: 'ageM'

In [14]:
dataf.head()

,usertype,birth year,gender,date
0,Subscriber,1991,1,2014-02-01 00:00:00
1,Subscriber,1979,2,2014-02-01 00:00:03
2,Subscriber,1948,2,2014-02-01 00:00:09
3,Subscriber,1981,1,2014-02-01 00:00:32
4,Subscriber,1990,1,2014-02-01 00:00:41


In [16]:
bins = np.arange(10, 99, 5)

axM = dataf.ageM.groupby(pd.cut(dataf.ageM, bins)).agg([count_nonzero]).plot(kind='bar', legend=False)
axM.set_title("male riders")

axF = dataf.ageF.groupby(pd.cut(dataf.ageF, bins)).agg([count_nonzero]).plot(kind='bar', legend=False)
axF.set_title("female riders")

AttributeError: 'DataFrame' object has no attribute 'ageM'

In [17]:
dataf.columns

Index([u'usertype', u'birth year', u'gender', u'date'], dtype='object')

In [18]:
dataf.head()

,usertype,birth year,gender,date
0,Subscriber,1991,1,2014-02-01 00:00:00
1,Subscriber,1979,2,2014-02-01 00:00:03
2,Subscriber,1948,2,2014-02-01 00:00:09
3,Subscriber,1981,1,2014-02-01 00:00:32
4,Subscriber,1990,1,2014-02-01 00:00:41
